In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import math
import sys
from collections import Counter

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import configuration_util
filePathDict = configuration_util.get_all_files_path_dict()
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import get_gene_and_cohort_list_utils

In [2]:
def summarize_double_hit_occurence_data(maf, lossAnnotatedMaf):
    tsgs = get_gene_and_cohort_list_utils.get_tsgs()
    driverMaf = maf[maf['isOncogenic'] == True]
    doubleHitValidMaf = driverMaf[(driverMaf['isDoubleHit'] == True) & (driverMaf['doubleHitValidates'] != 'other_SNPs_unbalanced')]
    counts = dict(doubleHitValidMaf['allele'].value_counts())
    listOfDicts = []
    for allele in set(doubleHitValidMaf['allele']):
        gene = allele.split('_')[0]
        doubleHitCount = counts[allele]
        geneMaf = driverMaf[driverMaf['Hugo_Symbol'] == gene]
        geneAlleleMaf = geneMaf[geneMaf['allele'] == allele]
        
        #note this does not include biallelic loss via 2x mutation
        nCasesBiallelicLoss = len(set(lossAnnotatedMaf[(lossAnnotatedMaf['lossType'] != False) & (lossAnnotatedMaf['Hugo_Symbol'] == gene)]['Tumor_Sample_Barcode']))
        
        geneType = 'TSG' if gene in tsgs else 'Oncogene'
        listOfDicts.append({
            'gene': gene, 'allele': allele, 'nGene': geneMaf.shape[0], 'nAllele': geneAlleleMaf.shape[0],
            'geneType': geneType, 'nDoubleHit': doubleHitCount, 'nBiallelicLoss': doubleHitCount + nCasesBiallelicLoss
        })
    return pd.DataFrame(listOfDicts)
        
        
        
        

In [2]:
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,17,18,40,41,43,44,71,108,124,125,128,129,130,131,132,133,134,135,137) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
allImpactMutsMaf['varUuid'] = allImpactMutsMaf.apply(lambda row:
    row['Tumor_Sample_Barcode'] + '_' + str(row['Start_Position']) + '_' + str(row['Tumor_Seq_Allele2']), axis=1)
oncogenicMutIds = set(allImpactMutsMaf[allImpactMutsMaf['oncogenic'].notnull()]['varUuid'])

mafWithClonalityAnnotation = pd.read_csv(filePathDict['IMPACT_MAF_WITH_ADJUSTED_CLONALITY_ANNOTATION'])
mafWithClonalityAnnotation['varUuid'] = mafWithClonalityAnnotation.apply(lambda row:
    row['Tumor_Sample_Barcode'] + '_' + str(row['Start_Position']) + '_' + str(row['Tumor_Seq_Allele2']), axis=1)


/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1,2,4,6,7,8,10,11,12,13,14,21,22,23,24,25,26,37,38,39,40,41,42,44,45,54,55,58,59,60,61,62,67,68,71,74,76,77,81,82,83,84,85,86,93,95,102,103,109,110,111,112,128,130,135,136,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
#mafWithClonalityAnnotation['isOncogenic'] = mafWithClonalityAnnotation['varUuid'].apply(lambda x: True if x in oncogenicMutIds else False)

In [12]:
reload(maf_analysis_utils)
flatGenomeCases = clonality_analysis_util.get_facets_whitelist()
hypermutantCases = get_gene_and_cohort_list_utils.get_all_hypermutant_ids()
hypermutantMaf = mafWithClonalityAnnotation[mafWithClonalityAnnotation['Tumor_Sample_Barcode'].isin(hypermutantCases)]
hypermutantMaf['flatGenome'] = hypermutantMaf['Tumor_Sample_Barcode'].apply(lambda x: True if x in flatGenomeCases else False)
hypermutantMaf = maf_analysis_utils.mark_cases_with_median_clonal_vaf_of_case(hypermutantMaf)


/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:173: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  df = pd.read_table(path)
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/maf_analysis_utils.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  vafMapping[case] = medianVaf


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/maf_analysis_utils.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  #BEST most efficient way to quickly count up the number of times a value occurs


In [31]:
hypermutantMaf['isDoubleHit'] = hypermutantMaf.apply(lambda row:
    clonality_analysis_util.is_mut_double_hit(row, row['flatGenome'], 
 doubleFactor=2), axis=1)
reload(clonality_analysis_util)
hypermutantMaf = clonality_analysis_util.annotate_double_hit_mutations(hypermutantMaf)

/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/clonality_analysis_util.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/clonality_analysis_util.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  #if (math.isnan(row['ccf_Mcopies']) or math.isnan(row['ccf_1copy'])) #dont analyze regions where Mcopies or 1copy are null (flat genome cases have already been called true)
/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts

In [71]:

driverMaf = hypermutantMaf[hypermutantMaf['isOncogenic'] == True]

driverMaf['geneCase'] = driverMaf.apply(lambda row: row['Hugo_Symbol'] + '_' + row['Tumor_Sample_Barcode'], axis=1)
occurenceDict = dict(driverMaf['geneCase'].value_counts())                                                                                     
driverMaf['isMultiplet'] = driverMaf['geneCase'].apply(lambda x: True if occurenceDict[x] > 1 else False)
driverMaf['isLOH'] = driverMaf['lcn'].apply(lambda x: True if x == 0 else False)
driverMaf['lossType'] = driverMaf.apply(lambda row: 'LOH' if row['isLOH'] == True
    else 'composite_mutation' if row['isMultiplet'] == True else False, axis=1)

driverMaf['allele'] = driverMaf.apply(lambda row: str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [74]:
df = summarize_double_hit_occurence_data(hypermutantMaf, driverMaf)

In [75]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/doubleHitPlot.tsv', index=False, sep='\t')